In [1]:
import sys
from pathlib import Path


def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False


def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book


def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('ccfraud',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")

Local environment
Added the following directory to the PYTHONPATH: /home/jdowling/Projects/mlfs-book/notebooks/ch06/cc-fraud


In [2]:
import polars as pl
import numpy as np
from datetime import datetime, timedelta
from faker import Faker
import random
import json
import ipaddress
from mlfs.config import settings
import hopsworks
import pandas as pd
import json


fake = Faker('en_US')

# Helper functions
def random_date_vec(start, end, size):
    """Generate a vector of random datetimes between `start` and `end`."""
    delta_seconds = (end - start).total_seconds()
    random_seconds = np.random.randint(0, int(delta_seconds), size)
    return pl.Series([start + timedelta(seconds=int(s)) for s in random_seconds])

def right_skewed_amount(size, mean=200, median=50):
    """Generate a right-skewed amount with mean and median."""
    return pl.Series(np.random.gamma(2, 100, size).clip(0.01, 10000).round(2))

def random_risk_score(size):
    """Generate right-skewed risk scores."""
    return pl.Series(np.random.beta(2, 8, size))


def random_amazon_ip():
    network = ipaddress.IPv4Network('3.0.0.0/8')
    return str(fake.ipv4(network=network))

def random_ip():
    return fake.ipv4()

def generate_ip():
    if random.choice([True, False]):
        return random_amazon_ip()  # 50% Amazon IP
    else:
        return random_ip()  

def is_amazon_ip(ip):
    network = ipaddress.IPv4Network('3.0.0.0/8')
    try:
        return ipaddress.IPv4Address(ip) in network
    except ipaddress.AddressValueError:
        return False

def generate_exponential_value(scale, max_value):
    value = np.random.exponential(scale)
    return min(value, max_value) 

# Generate Transactions Table
n_transactions = 500_000
n_cards = 10_000
n_customers = 8_000
n_merchants = 2_000
n_banks = 1_000
n_days = 10

now = datetime.now()

transaction_df = pl.DataFrame({
    "t_id": pl.Series([fake.uuid4() for _ in range(n_transactions)]),
    "cc_num": pl.Series(np.random.randint(1, n_cards, n_transactions).astype(str)),
    "merchant_id": pl.Series(np.random.randint(1, n_merchants, n_transactions).astype(str)),
    "amount": right_skewed_amount(n_transactions),
    "ip_address": pl.Series([generate_ip() for _ in range(n_transactions)]),
    "ts": random_date_vec(now - timedelta(days=n_days), now, n_transactions),
})

# card_present is computed using an existing column
transaction_df = transaction_df.with_columns( pl.col("ip_address").map_elements(is_amazon_ip).alias("card_present"))

# Generate Cards Table
issue_dates = random_date_vec(now - timedelta(days=5*365), now, n_cards)
expiry_dates = issue_dates + pl.Series(np.random.randint(3*365, 5*365, n_cards)).cast(pl.Duration)
card_df = pl.DataFrame({
    "cc_num": pl.Series(np.arange(1, n_cards+1).astype(str)),
    "account_id": pl.Series(np.random.randint(1, n_cards, n_cards).astype(str)),
    "bank_id": pl.Series(np.random.randint(1, n_banks, n_cards).astype(str)),
    "card_type": pl.Series(np.random.choice(['Credit', 'Debit', 'Prepaid'], n_cards, p=[0.6, 0.35, 0.05])),
    "issue_date": issue_dates,
    "expiry_date": expiry_dates,
    "status": pl.Series(np.random.choice(['Active', 'Blocked', 'Lost/Stolen'], n_cards, p=[0.95, 0.04, 0.01]))
})

# Generate Customers Table
customer_df = pl.DataFrame({
    "account_id": pl.Series(np.arange(1, n_customers+1).astype(str)),
    "name": pl.Series(fake.name() for _ in range(n_customers)),
    "address": pl.Series(fake.address() for _ in range(n_customers)),
    "email": pl.Series(fake.email() for _ in range(n_customers)),
    "creation_date": random_date_vec(now - timedelta(days=10*365), now, n_customers),
    "debt_end_prev_month": pl.Series(generate_exponential_value(0, 50000) for _ in range(n_customers)),
    "last_modified": random_date_vec(now - timedelta(days=30), now - timedelta(days=1), n_customers),
})

# Generate Merchants Table
merchant_df = pl.DataFrame({
    "merchant_id": pl.Series(np.arange(1, n_merchants+1).astype(str)),
    "merchant_name": pl.Series(fake.company() for _ in range(n_merchants)),
    "category": pl.Series(np.random.choice(['Grocery', 'Electronics', 'Clothing', 'Entertainment', 'Restaurants'], n_merchants)),
    "country": pl.Series(fake.country() for _ in range(n_merchants)),
    "last_modified": random_date_vec(now - timedelta(days=8), now - timedelta(days=7), n_merchants)
})

    # "cnt_chargeback_prev_week": right_skewed_amount(n_merchants, 100, 50),
    # "average_transaction_amount": right_skewed_amount(n_merchants, 100, 50)

# Generate Banks Table
bank_df = pl.DataFrame({
    "bank_id": pl.Series(np.arange(1, n_banks+1).astype(str)),
    "country": pl.Series(fake.country() for _ in range(n_banks)),
    "credit_rating": pl.Series(np.random.choice(['AAA', 'AA', 'A', 'BBB', 'BB', 'B', 'C'], n_banks)),
    "last_modified": random_date_vec(now - timedelta(days=7), now - timedelta(days=6), n_banks),
})

# Check time taken
time_taken = datetime.now() - now
print(f"Time taken={time_taken}")

2025-01-08 08:43:33,286 WARNING: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

Time taken=0:00:28.130054


In [3]:
transaction_df.head()

t_id,cc_num,merchant_id,amount,ip_address,ts,card_present
str,str,str,f64,str,datetime[μs],bool
"""98e0f222-d81c-4bac-9231-ef1a35…","""2207""","""949""",91.44,"""106.70.176.0/20""",2025-01-08 03:01:24.488169,false
"""1ec73941-acd4-4738-92f2-d0db9a…","""8621""","""1014""",104.08,"""217.87.192.66/31""",2025-01-01 14:26:43.488169,false
"""ef0bc66c-281a-48e5-b259-6d15fe…","""9076""","""1521""",70.9,"""13.159.10.29""",2025-01-01 23:09:31.488169,false
"""c0ee3edc-86bd-4d64-a995-093360…","""392""","""1819""",288.28,"""164.102.65.168/29""",2024-12-31 11:56:34.488169,false
"""3f47e776-b8fd-494a-b3af-dd6cff…","""5088""","""1650""",58.59,"""139.23.150.76/30""",2025-01-07 16:23:35.488169,false


In [ ]:
n_fraud_transactions = 5000

# Function to generate random IP addresses
def generate_fraud_ip():
    return f"{np.random.randint(0, 256)}.{np.random.randint(0, 256)}.{np.random.randint(0, 256)}.{np.random.randint(0, 256)}"

# Simulate right-skewed transaction amounts
def right_skewed_amount(n):
    return np.random.gamma(shape=2, scale=50, size=n)

# Generate random boolean values
def random_boolean(size=None):
    return np.random.choice([True, False], size=size)

# Generate random timestamps in a vectorized way
def random_date_vec(start, end, n):
    delta = (end - start).total_seconds()
    return [start + timedelta(seconds=np.random.randint(0, delta)) for _ in range(n)]

# Generate grouped transactions
def generate_grouped_transactions(n_transactions, min_group=4, max_group=10, window=30, two_ip=False, card_present_val=False):
    transactions = []
    current_time = now - timedelta(days=n_days)
    cc_nums = np.random.randint(1, n_cards, n_transactions).astype(str)
    
    while len(transactions) < n_fraud_transactions:
        group_size = np.random.randint(min_group, max_group + 1)
        group_size = min(group_size, n_transactions - len(transactions))
        
        base_time = current_time + timedelta(minutes=np.random.randint(0, window))
        time_offsets = np.random.normal(loc=15, scale=5, size=group_size)
        time_offsets = np.clip(time_offsets, 0, window * 60)
        cc_num = np.random.choice(cc_nums)
        
        for i, offset in enumerate(time_offsets):
            transactions.append({
                "t_id": fake.uuid4(),
                "cc_num": cc_num,
                "merchant_id": str(np.random.randint(1, n_merchants)),
                "amount": np.random.gamma(2, 50),
                "ip_address": generate_fraud_ip() if not two_ip or i % 2 == 0 else generate_fraud_ip(),
                "ts": base_time + timedelta(seconds=offset),
                "card_present": card_present_val if not two_ip else True
            })
        
        # Move to next window
        current_time += timedelta(minutes=window)
    
    return transactions

# Generate 5000 transactions (4-10 per group)
transactions_chain = generate_grouped_transactions(n_fraud_transactions, 4, 10, 30)

# Generate 2000 transactions (2 per group, different IPs, card_present=True)
transactions_loc = generate_grouped_transactions(n_fraud_transactions, 2, 2, 10, two_ip=True, card_present_val=True)

transactions_fraud = pl.DataFrame(transactions_chain + transactions_loc)
# Combine and create DataFrame
transactions_fraud.head()

In [ ]:
transaction_df = pl.concat([transaction_df, transactions_fraud])
print(transaction_df.head())

In [ ]:

# Generate Historical Transaction Fraud Reports Table
n_fraud_reports = int(n_fraud_transactions * 2)
fraud_df = pl.DataFrame({
    "report_id": pl.Series([fake.uuid4() for _ in range(n_fraud_reports)]),
    "t_id": transactions_fraud["t_id"],
    "report_time": random_date_vec(now, now + timedelta(days=1), n_fraud_reports),
    "fraud_type": pl.Series(np.random.choice(['Card Not Present', 'Skimming', 'Lost/Stolen'], n_fraud_reports, p=[0.5, 0.3, 0.2]))
})



In [ ]:
print(fraud_df)

In [ ]:
weekly_fraud = (
    transactions_fraud
    .with_columns(
        pl.col("ts").dt.week().alias("week"),
        pl.col("ts").dt.year().alias("year")
    )
    .group_by(["merchant_id", "year", "week"])
    .agg(
        pl.sum("amount").alias("total_fraud_amount")
    )
)

# Calculate total transactions per merchant per week and month
weekly_transactions = (
    transaction_df
    .with_columns(
        pl.col("ts").dt.week().alias("week"),
        pl.col("ts").dt.year().alias("year")
    )
    .group_by(["merchant_id", "year", "week"])
    .agg(
        pl.sum("amount").alias("total_amount")
    )
)

# Calculate fraud rate by week and month
weekly_fraud_rate = (
    weekly_fraud.join(
        weekly_transactions, on=["merchant_id", "year", "week"], how="inner"
    )
    .with_columns(
        (pl.col("total_fraud_amount") / pl.col("total_amount")).alias("weekly_fraud_rate")
    )
    .join(
        merchant_df, on="merchant_id", how="inner"
    )
    .filter(
        (pl.col("year") == pl.col("last_modified").dt.year()) &
        (pl.col("week") == pl.col("last_modified").dt.week())
    )
)

merchant_df = merchant_df.join(weekly_fraud_rate.select(["merchant_id", "weekly_fraud_rate"]), on="merchant_id", how="left")

nan_count = merchant_df["weekly_fraud_rate"].is_nan().sum()

print("NaN count in weekly_fraud_rate:", nan_count)

In [ ]:
merchant_df = merchant_df.with_columns(
    pl.col("weekly_fraud_rate").fill_null(0)
)
print(merchant_df)

In [ ]:
# Initialize Hopsworks connection and get a reference to the feature store using the project
if is_google_colab():
    project = hopsworks.login() # prompts the user to enter their HOPSWORKS_API_KEY in the colab notebook
else:
    project = hopsworks.login(api_key_value=settings.HOPSWORKS_API_KEY.get_secret_value()) # read HOPSWORKS_API_KEY from ~/.env file

fs = project.get_feature_store()


In [ ]:
# Define descriptions for each table schema's features
feature_descriptions = {
    "credit_card_transactions": {
        "t_id": "Unique identifier for each transaction.",
        "cc_num": "Identifier for the card used in the transaction.",
        "merchant_id": "Identifier for the merchant where the transaction took place.",
        "ts": "Timestamp for when the transaction occurred.",
        "amount": "Monetary amount of the transaction, right-skewed, median $50, mean $200.",
        "ip_address": "IP address for  the credit terminal that executed the transaction",
        "card_present": "Point-of-sale terminal (true) or an online transaction (false)."
    },
    "card_details": {
        "cc_num": "Unique identifier for each card.",
        "account_id": "Owner of the card.",
        "bank_id": "Issuer of the card.",
        "card_type": "Type of card ('Credit', 'Debit', 'Prepaid').",
        "issue_date": "Date when the card was issued.",
        "expiry_date": "Card's expiration date, typically 3 to 5 years after issue.",
        "status": "Current status of the card ('Active', 'Blocked', 'Lost/Stolen')."
    },
    "account_details": {
        "account_id": "Unique identifier for each customer.",
        "name": "First and last name of the customer.",
        "email": "Email address of the customer.",
        "address": "Address of the customer.",
        "debt_end_prev_month": "Outstanding credit card debt at end of most recent month",        
        "creation_date": "Date when the customer joined.",
        "last_modified": "Timestamp of last update for this row."
    },
    "merchant_details": {
        "merchant_id": "Unique identifier for each merchant.",
        "merchant_name": "Name of the merchant.",
        "category": "Category of the merchant (e.g., 'Grocery', 'Electronics').",
        "country": "Location of the merchant.",
        "last_modified": "Timestamp of last update for this row."
    },
    "bank_details": {
        "bank_id": "Unique identifier for each merchant.",
        "country": "Home country for the bank.",
        "credit_rating": "Bank credit rating (from AAA to C)",
        "last_modified": "Timestamp of last update for this row."
    },
    "cc_fraud": {
        "report_id": "Unique identifier for each fraud report.",
        "t_id": "The transaction reported as fraudulent.",
        "report_time": "Timestamp when the fraud was reported.",
        "fraud_type": "Type of fraud ('Card Not Present', 'Skimming', 'Lost/Stolen')."
    },
}

# Function to create and insert DataFrame into a feature group
def create_feature_group(feature_group_name, description, df, feature_descriptions, event_time, online, wait):
    # Create feature group in the Hopsworks feature store
    feature_group = fs.get_or_create_feature_group(
        name=feature_group_name,
        version=1,
        description=description,
        primary_key=list(df.columns[:1]),
        event_time=event_time,
        online_enabled=online
    )
    
    # Insert data into the feature group
    feature_group.insert(df, wait=wait)
    
    # Update feature descriptions
    for feature, desc in feature_descriptions.items():
        feature_group.update_feature_description(feature, desc)


In [ ]:
# Create and insert data into feature groups
create_feature_group(
    "credit_card_transactions",
    "Records of all financial transactions made using cards.",
    transaction_df,
    feature_descriptions["credit_card_transactions"],
    "ts",
    False,
    False
)


In [ ]:
create_feature_group(
    "card_details",
    "Information about the cards used by customers.",
    card_df,
    feature_descriptions["card_details"],
    None,
    False,
    False
)


In [ ]:

create_feature_group(
    "account_details",
    "Personal information about the customer acoounts.",
    customer_df,
    feature_descriptions["account_details"],
    None,
    False,
    False
)


In [ ]:

create_feature_group(
    "merchant_details",
    "Details about merchants who execute credit card transactions.",
    merchant_df,
    feature_descriptions["merchant_details"],
    None,
    False,
    False    
)



In [ ]:

create_feature_group(
    "bank_details",
    "Details about banks - credit card issuers.",
    bank_df,
    feature_descriptions["bank_details"],
    None,
    False,
    True    
)



In [ ]:
create_feature_group(
    "cc_fraud",
    "Records of past fraudulent activities reported.",
    fraud_df,
    feature_descriptions["cc_fraud"],
    "report_time",
    False,
    False    
)



In [ ]:
# import json
# import os
# from confluent_kafka import Producer

In [ ]:
# KAFKA_TOPIC_NAME = f"{project.name}_real_time_live_transactions"
# SCHEMA_NAME = "live_transactions_schema"
# kafka_api = project.get_kafka_api()

In [ ]:
# schema = {
#     "type": "record",
#     "name": SCHEMA_NAME,
#     "namespace": "ai.hopsworks.examples.feldera.fraud",
#     "fields": [
#         {
#             "name": "t_id",
#             "type": [
#                 "null",
#                 "string"
#             ]
#         },
#         {
#             "name": "ts",
#             "type": [
#                 "null",
#                 {
#                     "type": "long",
#                     "logicalType": "timestamp-micros"
#                 }
#             ]
#         },
#         {
#             "name": "cc_num",
#             "type": [
#                 "null",
#                 "string"
#             ]
#         },
#         {
#             "name": "merchant_id",
#             "type": [
#                 "null",
#                 "string"
#             ]
#         },
#         {
#             "name": "amount",
#             "type": [
#                 "null",
#                 "double"
#             ]
#         },
#         {
#             "name": "ip_addr",
#             "type": [
#                 "null",
#                 "string"
#             ]
#         },
#         {
#             "name": "card_present",
#             "type": [
#                 "null",
#                 "boolean"
#             ]
#         }
#     ]
# }

# if KAFKA_TOPIC_NAME not in [topic.name for topic in kafka_api.get_topics()]:
#     kafka_api.create_schema(SCHEMA_NAME, schema)
#     kafka_api.create_topic(KAFKA_TOPIC_NAME, SCHEMA_NAME, 1, replicas=1, partitions=1)
#     print("Created topic")
# else:
#     print("Did not create topic")

In [ ]:
# # Uncomment and run this cell to delete the schema and topic

# topics = kafka_api.get_topics()
# if topics[0].name == KAFKA_TOPIC_NAME:
#     try:
#         topics[0].delete()
#         print(f"Deleted topic {KAFKA_TOPIC_NAME}")
#     except hopsworks.RestAPIError:
#         print("Could not find topic to delete")

# try:
#     schema = kafka_api.get_schema(SCHEMA_NAME, 1)
#     print(f"Deleting schema {SCHEMA_NAME}")
#     schema.delete()
# except hopsworks.RestAPIError:
#     print("Could not find schema to delete")

In [ ]:
# # Get kafka config, so that you can produce to the Topic
# kafka_config = fs._storage_connector_api.get_kafka_connector(fs.id, True).confluent_options()

# print(kafka_config)
# producer = Producer(kafka_config)

# for index, transaction in transaction_df.iterrows():
#     producer.produce(KAFKA_TOPIC_NAME, transaction.to_json())
    
#     if index % 50000 == 0:
#         producer.flush()
#         print(f'Finished sending index {index}')

# producer.flush()